## MOVIE RECOMMENDATION


**IMPORTING LIBRARIES**

In [1]:
import numpy as np
import pandas as pd
import ast
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

**OPENING THE FILES**

In [2]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv',error_bad_lines=False, engine="python")

<ipython-input-2-891651187b64>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  credits = pd.read_csv('tmdb_5000_credits.csv',error_bad_lines=False, engine="python")
Skipping line 171: unexpected end of data


In [3]:
#Merging the datasets
movies = movies.merge(credits, on = 'title')


In [4]:
movies.shape

(169, 23)

In [5]:
#Importing necessary columns
movies = movies[['movie_id','title','genres','overview','keywords','cast','crew']]
movies.head(1)

,movie_id,title,genres,overview,keywords,cast,crew
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","In the 22nd century, a paraplegic Marine is di...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


**DATA PREPROCESSING**

In [6]:
movies.isnull().sum()

movie_id    0
title       0
genres      0
overview    0
keywords    0
cast        0
crew        0
dtype: int64

In [7]:
movies.dropna(inplace = True)

In [8]:
movies.duplicated().sum()

0

In [9]:
def convert(obj):
  L = []
  for i in ast.literal_eval(obj):
    L.append(i['name'])
  return L


In [10]:
movies['genres'] = movies['genres'].apply(convert)

In [11]:
movies['keywords'] = movies['keywords'].apply(convert)

In [12]:
def convert3(obj):
  L = []
  counter = 0
  for i in ast.literal_eval(obj):
    if counter<3:
      L.append(i['name'])
      counter+=1
    else:
      break
  return L

In [13]:
movies['cast'] = movies['cast'].apply(convert3)
movies.head(1)

,movie_id,title,genres,overview,keywords,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","In the 22nd century, a paraplegic Marine is di...","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [14]:
def fetchdirector(obj):
  L = []
  for i in ast.literal_eval(obj):
    if i['job'] == 'Director':
      L.append(i['name'])
    else:
      break
  return L

In [15]:
 movies['crew'] = movies['crew'].apply(fetchdirector)

In [16]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())
movies.head()

,movie_id,title,genres,overview,keywords,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[In, the, 22nd, century,, a, paraplegic, Marin...","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[]
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","[Captain, Barbossa,, long, believed, to, be, d...","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[]
2,206647,Spectre,"[Action, Adventure, Crime]","[A, cryptic, message, from, Bond’s, past, send...","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[]
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]","[Following, the, death, of, District, Attorney...","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman]",[]
4,49529,John Carter,"[Action, Adventure, Science Fiction]","[John, Carter, is, a, war-weary,, former, mili...","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton]",[]


In [17]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(' ','') for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(' ','')  for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(' ','')  for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[[i.replace(' ','')  for i in x]])
movies.head()

,movie_id,title,genres,overview,keywords,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction]","[In, the, 22nd, century,, a, paraplegic, Marin...","[[cultureclash, future, spacewar, spacecolony,...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[]
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","[Captain, Barbossa,, long, believed, to, be, d...","[[ocean, drugabuse, exoticisland, eastindiatra...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[]
2,206647,Spectre,"[Action, Adventure, Crime]","[A, cryptic, message, from, Bond’s, past, send...","[[spy, basedonnovel, secretagent, sequel, mi6,...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[]
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]","[Following, the, death, of, District, Attorney...","[[dccomics, crimefighter, terrorist, secretide...","[ChristianBale, MichaelCaine, GaryOldman]",[]
4,49529,John Carter,"[Action, Adventure, ScienceFiction]","[John, Carter, is, a, war-weary,, former, mili...","[[basedonnovel, mars, medallion, spacetravel, ...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[]


In [18]:
#Creating tags columns containing important information
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']
movies.head()

,movie_id,title,genres,overview,keywords,cast,crew,tags
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction]","[In, the, 22nd, century,, a, paraplegic, Marin...","[[cultureclash, future, spacewar, spacecolony,...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[],"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","[Captain, Barbossa,, long, believed, to, be, d...","[[ocean, drugabuse, exoticisland, eastindiatra...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[],"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[Action, Adventure, Crime]","[A, cryptic, message, from, Bond’s, past, send...","[[spy, basedonnovel, secretagent, sequel, mi6,...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[],"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]","[Following, the, death, of, District, Attorney...","[[dccomics, crimefighter, terrorist, secretide...","[ChristianBale, MichaelCaine, GaryOldman]",[],"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[Action, Adventure, ScienceFiction]","[John, Carter, is, a, war-weary,, former, mili...","[[basedonnovel, mars, medallion, spacetravel, ...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[],"[John, Carter, is, a, war-weary,, former, mili..."


In [19]:
#dataset sorted
new_df = movies[['movie_id','title','tags']]
new_df.head()

,movie_id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."


In [20]:
new_df['tags'] = new_df['tags'].apply(lambda x:' '.join(map(str,x)))

<ipython-input-20-dd26be9503d3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:' '.join(map(str,x)))


In [21]:
new_df.head()

,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


In [22]:
!pip install nltk

In [23]:
#Applying stemming
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()


In [24]:
def stem(text):
  y = []
  for i in text.split():
    y.append(ps.stem(i))

  return " ".join(y)

In [25]:
new_df['tags'] = new_df['tags'].apply(stem)

<ipython-input-25-be18a4346d89>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


In [26]:
  new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

<ipython-input-26-8b60b591a07f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())


In [27]:
new_df.head()

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a parapleg marin is dispa..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believ to be dead, ha c..."
2,206647,Spectre,a cryptic messag from bond’ past send him on a...
3,49026,The Dark Knight Rises,follow the death of district attorney harvey d...
4,49529,John Carter,"john carter is a war-weary, former militari ca..."


In [28]:
#Creating vectors
cv = CountVectorizer(max_features=5000, stop_words = 'english')
vectors = cv.fit_transform(new_df['tags']).toarray()

In [29]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zombies', 'zon', 'zoo'], dtype=object)

In [30]:
 similarity = cosine_similarity(vectors)

**Function to recommend similar movies**

In [31]:
def recommend(movie):
  movie_index = new_df[new_df['title'] == movie].index[0]
  distances = similarity[movie_index]
  movie_list = sorted(list(enumerate(distances)), reverse = True, key = lambda x:x[1])[1:6]
  for i in movie_list:
    print(new_df.iloc[i[0]].title)

In [32]:
recommend('Batman Begins')

The Dark Knight
The Dark Knight Rises
Batman v Superman: Dawn of Justice
Iron Man 2
The Amazing Spider-Man
